Price forecasting

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from VMD import VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
K =2             # 3 modes  
DC = 0             # no DC part imposed  
init = 1           # initialize omegas uniformly  
tol = 1e-7  

import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import( LinearRegression, 
                                 ElasticNet, 
                                 Ridge, 
                                 Lasso, 
                                 HuberRegressor)
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
gold_prices=pd.read_csv("gold.csv") #load data
varpre=pd.read_csv('shuju.csv') #REPRESENT VAR model prediction results
gold_prices=gold_prices.iloc[:,0:]
gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
gold_prices.set_index('Period',inplace=True)
with pd.ExcelWriter('DEprice-proposed-mult_sheets.xlsx') as writer:
    for r in range(10):  # 
      
        def create_lag_features(data, lag):
            lagged_data = pd.DataFrame(index=data.index)
            for col in data.columns:
                if col != 'Period':
                    for k in range(1, lag + 1):
                        lagged_data[f'{col}_Lag_{k}'] = data[col].shift(k)
#                     lagged_data=lagged_data.drop(f'{col}_Lag_1',axis=1)
            return lagged_data


        lag = 3 # lag
        # features = create_lag_features(gold_prices, lag)
        feature = gold_prices

        window_size = 44  # window size

        # initionlzation
        modelss = {
            'TrendModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'SeasonalityModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'ResidualModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
                'TrendModel3':RandomForestRegressor(),
            'SeasonalityModel3': RandomForestRegressor(),
            'ResidualModel3': RandomForestRegressor(),
            'TrendModel4': LinearRegression(),
            'SeasonalityModel4': LinearRegression(),
            'ResidualModel4': LinearRegression()
        #         'Lasso': Lasso(0.5, fit_intercept=False)
        }

        # train and predict
        # predictions = {model_name: [] for model_name in models}
        actual_price = []
        direction_correct = 0
        predictions=[]
        # features=features.fillna(method='bfill')
        # features=features.dropna()
        # print(features)
        for i in range(len(gold_prices['GoldPrice']) - window_size):

            # STL decomposition
            stl = STL(gold_prices['GoldPrice'].iloc[0:i + window_size])
            result = stl.fit()
            trend = result.trend
            seasonalo = result.seasonal
            residualo = result.resid
            newdata=seasonalo+residualo
            u, u_hat, omega = VMD(newdata, alpha, tau, K, DC, init, tol)
            u=u.astype(float)
            seasonal = pd.DataFrame(u[1],index=gold_prices.index[0:(i + window_size)])
            seasonal.columns = ['seasonal']
            residual = pd.DataFrame(u[0],index=gold_prices.index[0:(i + window_size)])
            residual.columns = ['residual']


            feature['Trend'] = trend
            feature['Seasonal'] = seasonal
            feature['Residual'] = residual
            #create after features
        #     features = create_lag_features(feature, lag)
        #     names=['GoldPrice','TrendModel','SeasonalityModel','ResidualModel']
        #     features[names] = feature[names]
        #     print(feature.shape)

        #    '''delete factors'''
            featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
            featureseasonal=feature.drop(['Trend','Residual'], axis=1)
            featureresidual=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Trend'], axis=1)
            
#             featuretrend=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
#             featureseasonal=feature.drop(['Trend','Residual'], axis=1)
#             featureresidual=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Trend'], axis=1)

            modelt=ARIMA(trend[0:i + window_size], order=(0, 1, 1))
            resultt = modelt.fit()
            trend_pred1 = resultt.forecast()[0]

            featuretrend = create_lag_features(featuretrend, lag)
            featuretrend['Trend']=feature['Trend']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtraint=featuretrend.iloc[lag:i + window_size,0:-1]
            ytraint=featuretrend.iloc[lag:i + window_size,-1]
            xtestt=featuretrend.iloc[i+window_size,0:-1]
        #     xtestt=np.array(xtestt).reshape(-1,featuretrend.shape[1]-1)

            featureseasonal = create_lag_features(featureseasonal, lag)
            featureseasonal['Seasonal']=feature['Seasonal']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrains=featureseasonal.iloc[lag:i + window_size,0:-1]
            ytrains=featureseasonal.iloc[lag:i + window_size,-1]
            xtests=featureseasonal.iloc[i+window_size,0:-1]
        #     xtests=np.array(xtests).reshape(-1,featureseasonal.shape[1]-1)

            models=ARIMA(residual[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            residual_pred1 = results.forecast()[0]

            featureresidual = create_lag_features(featureresidual, lag)
            featureresidual['Residual']=feature['Residual']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrainr=featureresidual.iloc[lag:i + window_size,0:-1]
            ytrainr=featureresidual.iloc[lag:i + window_size,-1]
            xtestr=featureresidual.iloc[i+window_size,0:-1]
        #     xtestr=np.array(xtestr).reshape(-1,featureresidual.shape[1]-1)

        #     scaler = MinMaxScaler()
        # #             scaler = StandardScaler()
        #     X_train = scaler.fit_transform(X_train)
        #     X_test = scaler.transform(X_test)
        #     scaler1 = MinMaxScaler()
        #     y_trend = scaler1.fit_transform(y_trend.values.reshape(-1, 1)).flatten()
        #     y_seasonal = scaler1.transform(y_seasonal.values.reshape(-1, 1)).flatten()
        #     y_residual = scaler1.transform(y_residual.values.reshape(-1, 1)).flatten()
            modelr=ARIMA(seasonal[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            resultr = modelr.fit()
            seasonal_pred1 = resultr.forecast()[0]

#             
#             modelfs = RandomForestRegressor(random_state=10)
#             
#             modelfs.fit(xtraint,ytraint)
#            
#             importances = modelfs.feature_importances_
#       
#             threshold = np.median(importances)
#             
#             selected_features = xtraint.columns[importances > threshold]
#             xtraint=xtraint[selected_features]
#             xtestt=xtestt[selected_features]
            xtestt=np.array(xtestt).reshape(-1,xtraint.shape[1])

                
#             modelfsr = RandomForestRegressor(random_state=10)
#            
#             modelfsr.fit(xtrainr,ytrainr)
#             
#             importancesr = modelfsr.feature_importances_
#            
#             thresholdr = np.median(importancesr)
#       
#             selected_featuresr = xtrainr.columns[importancesr > thresholdr]
#             xtrainr=xtrainr[selected_featuresr]
#             xtestr=xtestr[selected_featuresr]
            xtestr=np.array(xtestr).reshape(-1,xtrainr.shape[1])  

                    # create RF model
#             modelfss = RandomForestRegressor(random_state=10)
#             # 
#             modelfss.fit(xtrains,ytrains)
#             # 
#             importancess = modelfss.feature_importances_
#             # 
#             thresholds = np.median(importancess)
#           
#             selected_featuress = xtrains.columns[importancess > thresholds]
#             xtrains=xtrains[selected_featuress]
#             xtests=xtests[selected_featuress]
            xtests=np.array(xtests).reshape(-1,xtrains.shape[1])       


            modelss['SeasonalityModel'].fit(xtrains, ytrains)
            modelss['TrendModel'].fit(xtraint, ytraint)
            modelss['ResidualModel'].fit(xtrainr, ytrainr)

            # forecasting

            trend_pred2 = modelss['TrendModel'].predict(xtestt)
            seasonal_pred2 = modelss['SeasonalityModel'].predict(xtests)
            residual_pred2 = modelss['ResidualModel'].predict(xtestr)

            modelss['SeasonalityModel3'].fit(xtrains, ytrains)
            modelss['TrendModel3'].fit(xtraint, ytraint)
            modelss['ResidualModel3'].fit(xtrainr, ytrainr)

#             modelss['SeasonalityModel4'].fit(xtrains, ytrains)
#             modelss['TrendModel4'].fit(xtraint, ytraint)
#             modelss['ResidualModel4'].fit(xtrainr, ytrainr)
            # 进行预测

            trend_pred3 = modelss['TrendModel3'].predict(xtestt)
            seasonal_pred3 = modelss['SeasonalityModel3'].predict(xtests)
            residual_pred3 = modelss['ResidualModel3'].predict(xtestr)
            
#             trend_pred4 = modelss['TrendModel4'].predict(xtestt)
#             seasonal_pred4 = modelss['SeasonalityModel4'].predict(xtests)
#             residual_pred4 = modelss['ResidualModel4'].predict(xtestr)

            #inverse normalzation
        #     trend_pred = scaler1.inverse_transform(np.array(trend_pred).reshape(-1, 1)).flatten()
        #     seasonal_pred = scaler1.inverse_transform(np.array(seasonal_pred).reshape(-1, 1)).flatten()
        #     residual_pred = scaler1.inverse_transform(np.array(residual_pred).reshape(-1, 1)).flatten()

          
#             predicted = trend_pred2+ seasonal_pred1 + residual_pred2
            predicted = (trend_pred2+trend_pred3)/2 +seasonal_pred1 + (residual_pred2+residual_pred3)/2

#             predicted = (trend_pred2+ seasonal_pred1 + residual_pred2+trend_pred1+ seasonal_pred1 + residual_pred1+trend_pred3+ seasonal_pred1 + residual_pred3)/3
            actual_price.append(gold_prices['GoldPrice'].iloc[i + window_size])
            predicted=0.5*predicted+0.5*varpre.iloc[i,1]   # average weighting ensemble no dynamic weighting ensemble
       
            if i < len(feature) - window_size:
                    actual_change = gold_prices['GoldPrice'].iloc[i + window_size ] - gold_prices['GoldPrice'].iloc[i + window_size -1]
                    predicted_change = predicted - gold_prices['GoldPrice'].iloc[i + window_size -1]
                    if np.sign(actual_change) == np.sign(predicted_change):
                        direction_correct += 1

            predictions.append(predicted)
        predictions = np.concatenate([arr for arr in predictions])
        actual_price = np.array(actual_price)
        # coputing metrics
        mse = mean_squared_error(actual_price, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual_price, predictions)
        mape = np.mean(np.abs((actual_price - predictions) / actual_price)) * 100
        total_directions = len(feature) - window_size 
  
        direction_accuracy = (direction_correct/ total_directions) * 100

        metrics = {
                'RMSE': rmse,
                'MSE': mse,
                'MAE': mae,
                'MAPE':mape,
                'Directional Accuracy': direction_accuracy,
                'actual_price':actual_price,
                'predictions':predictions
            }
        print(metrics)


        # df_metrics = pd.DataFrame(metrics)
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        print(actual_price)
        print(predictions)
        df_metrics = pd.DataFrame(metrics)
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}', index=metrics)

Demand forecasting

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from VMD import VMD  
#. some sample parameters for VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
K =3             # 3 modes  
DC = 0             # no DC part imposed  
init = 1           # initialize omegas uniformly  
tol = 1e-7  

import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import( LinearRegression, 
                                 ElasticNet, 
                                 Ridge, 
                                 Lasso, 
                                 HuberRegressor)
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neural_network import MLPRegressor

gold_prices=pd.read_csv("gold.csv")
varpre=pd.read_csv('shuju.csv')
gold_prices=gold_prices.iloc[:,0:]
gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
gold_prices.set_index('Period',inplace=True)
with pd.ExcelWriter('DEdemand-Proposed-baggingarimamult_sheets.xlsx') as writer:
    for r in range(10):  # 

        def create_lag_features(data, lag):
            lagged_data = pd.DataFrame(index=data.index)
            for col in data.columns:
                if col != 'Period':
                    for k in range(1, lag + 1):
                        lagged_data[f'{col}_Lag_{k}'] = data[col].shift(k)
        #             lagged_data=lagged_data.drop(f'{col}_Lag_1',axis=1)
            return lagged_data

        lag = 3 # set lag
        # features = create_lag_features(gold_prices, lag)
        feature = gold_prices

        window_size = 44  # window size

        # initlazation model
        modelss = {
            'TrendModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'SeasonalityModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'ResidualModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'lasso1':RandomForestRegressor(),
            'lasso2': RandomForestRegressor(),
            'lasso3':RandomForestRegressor()
        }

        #train and predict
        # predictions = {model_name: [] for model_name in models}
        actual_price = []
        direction_correct = 0
        predictions=[]
        # features=features.fillna(method='bfill')
        # features=features.dropna()
        # print(features)
        for i in range(len(gold_prices['GoldPricede']) - window_size):

            # STL
            u, u_hat, omega = VMD(gold_prices['GoldPricede'].iloc[0:i + window_size], alpha, tau, K, DC, init, tol)
            u=u.astype(float)
            trend = pd.DataFrame(u[0],index=gold_prices.index[0:(i + window_size)])
            trend.columns = ['trend']

            newdata = pd.DataFrame(gold_prices['GoldPricede'].iloc[0:i + window_size]-u[0],index=gold_prices.index[0:(i + window_size)])
            stl = STL(newdata)
            result = stl.fit()
            seasonal = result.seasonal+result.resid
            residual = result.trend


            feature['Trend'] = trend
            feature['Seasonal'] = seasonal
            feature['Residual'] = residual
            #create after features
        #     features = create_lag_features(feature, lag)
        #     names=['GoldPrice','TrendModel','SeasonalityModel','ResidualModel']
        #     features[names] = feature[names]
        #     print(feature.shape)

        #     print(features.shape)
            featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
            featureseasonal=feature.drop(['GoldPricede','MAdemand','diffdemand','Trend','Residual'], axis=1)
            featureresidual=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Trend'], axis=1)
            #'''no external factors results'''
#             featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
#             featureseasonal=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Trend','Residual'], axis=1)
#             featureresidual=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Trend'], axis=1)

            modelt=ARIMA(trend[0:i + window_size], order=(0, 1, 1))
            resultt = modelt.fit()
            trend_pred1 = resultt.forecast()[0]

            featuretrend = create_lag_features(featuretrend, lag)
            featuretrend['Trend']=feature['Trend']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtraint=featuretrend.iloc[lag:i + window_size,0:-1]
            ytraint=featuretrend.iloc[lag:i + window_size,-1]
            xtestt=featuretrend.iloc[i+window_size,0:-1]
            xtestt=np.array(xtestt).reshape(-1,featuretrend.shape[1]-1)

            featureseasonal = create_lag_features(featureseasonal, lag)
            featureseasonal['Seasonal']=feature['Seasonal']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrains=featureseasonal.iloc[lag:i + window_size,0:-1]
            ytrains=featureseasonal.iloc[lag:i + window_size,-1]
            xtests=featureseasonal.iloc[i+window_size,0:-1]
        #     xtests=np.array(xtests).reshape(-1,featureseasonal.shape[1]-1)

            models=ARIMA(seasonal[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            seasonal_pred1 = results.forecast()[0]

            featureresidual = create_lag_features(featureresidual, lag)
            featureresidual['Residual']=feature['Residual']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrainr=featureresidual.iloc[lag:i + window_size,0:-1]
            ytrainr=featureresidual.iloc[lag:i + window_size,-1]
            xtestr=featureresidual.iloc[i+window_size,0:-1]


            models=ARIMA(residual[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            residual_pred1 = results.forecast()[0]

        #     xtestr=np.array(xtestr).reshape(-1,featureresidual.shape[1]-1)

        #     scaler = MinMaxScaler()
        # #             scaler = StandardScaler()
        #     X_train = scaler.fit_transform(X_train)
        #     X_test = scaler.transform(X_test)
        #     scaler1 = MinMaxScaler()
        #     y_trend = scaler1.fit_transform(y_trend.values.reshape(-1, 1)).flatten()
        #     y_seasonal = scaler1.transform(y_seasonal.values.reshape(-1, 1)).flatten()
        #     y_residual = scaler1.transform(y_residual.values.reshape(-1, 1)).flatten()

           
            modelfs = RandomForestRegressor(random_state=0)
          
            modelfs.fit(xtrains,ytrains)
           
            importances = modelfs.feature_importances_
            
            threshold = np.median(importances)
            
            selected_features = xtrains.columns[importances > threshold]
            xtrains=xtrains[selected_features]
            xtests=xtests[selected_features]
            xtests=np.array(xtests).reshape(-1,xtrains.shape[1])

     
            modelfsr = RandomForestRegressor(random_state=0)
     
            modelfsr.fit(xtrainr,ytrainr)

            importancesr = modelfsr.feature_importances_
          
            thresholdr = np.median(importances)
        
            selected_featuresr = xtrainr.columns[importancesr > thresholdr]
            xtrainr=xtrainr[selected_featuresr]
            xtestr=xtestr[selected_featuresr]
            xtestr=np.array(xtestr).reshape(-1,xtrainr.shape[1])  

            modelss['SeasonalityModel'].fit(xtrains, ytrains)
            modelss['TrendModel'].fit(xtraint, ytraint)
            modelss['ResidualModel'].fit(xtrainr, ytrainr)

            modelss['lasso2'].fit(xtrains, ytrains)
            modelss['lasso1'].fit(xtraint, ytraint)
            modelss['lasso3'].fit(xtrainr, ytrainr)
  

            trend_pred2 = modelss['TrendModel'].predict(xtestt)
            seasonal_pred2 = modelss['SeasonalityModel'].predict(xtests)
            residual_pred2 = modelss['ResidualModel'].predict(xtestr)
            
            trend_pred3 = modelss['lasso1'].predict(xtestt)
            seasonal_pred3 = modelss['lasso2'].predict(xtests)
            residual_pred3 = modelss['lasso3'].predict(xtestr)


        #     trend_pred = scaler1.inverse_transform(np.array(trend_pred).reshape(-1, 1)).flatten()
        #     seasonal_pred = scaler1.inverse_transform(np.array(seasonal_pred).reshape(-1, 1)).flatten()
        #     residual_pred = scaler1.inverse_transform(np.array(residual_pred).reshape(-1, 1)).flatten()

            predicted = trend_pred1+ seasonal_pred2 + residual_pred2
#             predicted = (trend_pred1+ seasonal_pred2 + residual_pred2+trend_pred1+ seasonal_pred3 + residual_pred3)/2

        
            actual_price.append(gold_prices['GoldPricede'].iloc[i + window_size])
            predicted=0.9*predicted+0.1*varpre.iloc[i,2]    
       
            if i < len(feature) - window_size:
                    actual_change = gold_prices['GoldPricede'].iloc[i + window_size ] - gold_prices['GoldPricede'].iloc[i + window_size -1]
                    predicted_change = predicted - gold_prices['GoldPricede'].iloc[i + window_size -1]
                    if np.sign(actual_change) == np.sign(predicted_change):
                        direction_correct += 1

            predictions.append(predicted)
        predictions = np.concatenate([arr for arr in predictions])
        actual_price = np.array(actual_price)
        # computing metrics
        mse = mean_squared_error(actual_price, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual_price, predictions)
        mape = np.mean(np.abs((actual_price - predictions) / actual_price)) * 100
        total_directions = len(feature) - window_size 
      
        direction_accuracy = (direction_correct/ total_directions) * 100

        metrics = {
                'RMSE': rmse,
                'MSE': mse,
                'MAE': mae,
                'MAPE':mape,
                'Directional Accuracy': direction_accuracy,
                'actual_price':actual_price,
                'predictions':predictions
            }
        print(metrics)

       
        # df_metrics = pd.DataFrame(metrics)
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        print(actual_price)
        print(predictions)
        df_metrics = pd.DataFrame(metrics)
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}', index=metrics)

VAR model 

In [ ]:
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller
import numpy as np
df=pd.read_csv("gold.csv")
window_size=int(df.shape[0]*0.8)-1   #window-size 
shuju=np.zeros((len(df)-window_size,2)) #
for i in range(len(df)-window_size):
    dfnew=df.iloc[0:window_size+i,0:3]  #
#     dfnew=df.iloc[0:window_size+i,0:3]#
    dfnew['Period']=pd.to_datetime(dfnew['Period'])
    dfnew.set_index('Period',inplace=True)
    data=dfnew
  
    def check_stationarity(data):
        for column in data.columns:
            result = adfuller(data[column])
            print(f'ADF Statistic for {column}: {result[0]}')
            print(f'p-value: {result[1]}')
            print('Critical Values:')
            for key, value in result[4].items():
                print(f'   {key}: {value}')

    check_stationarity(data)


    if not all(data.apply(lambda x: adfuller(x)[1] < 0.05)):
        data = data.diff().dropna()
        print("one order diff tackle：")
        print(data)
    
    # VAR model
    model = VAR(data)
    #
    lag_order=3
    result = model.fit()
    # forecasting
    forecast = result.forecast(data.values[-result.k_ar:], steps=1)
    # DataFrame
    forecast_df = pd.DataFrame(forecast, index=[data.index[-1] + pd.DateOffset(months=3)], columns=data.columns)
    print("prediction results：",forecast_df)
    shuju[i,0]=forecast_df.iloc[0,0]
    shuju[i,1]=forecast_df.iloc[0,1]
# shuju=pd.DataFrame(shuju)
# shuju.index=df['Period'].iloc[window_size+1:]
varpre=shuju+df.iloc[window_size-1:-1,1:3]
shuju=pd.DataFrame(shuju)
varpre.index=df['Period'].iloc[window_size:]
# varpre.to_csv('varpre-1step12test.csv')

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from VMD import VMD  
#. some sample parameters for VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
K =2             # 3 modes  
DC = 0             # no DC part imposed  
init = 1           # initialize omegas uniformly  
tol = 1e-7  

import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import( LinearRegression, 
                                 ElasticNet, 
                                 Ridge, 
                                 Lasso, 
                                 HuberRegressor)
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
gold_prices=pd.read_csv("gold.csv")
varpre=pd.read_csv('varpre-1step12test.csv')
gold_prices=gold_prices.iloc[:,0:]
gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
gold_prices.set_index('Period',inplace=True)
with pd.ExcelWriter('price-proposed-method.xlsx') as writer:
    for r in range(10):  #
        # 
        def create_lag_features(data, lag):
            lagged_data = pd.DataFrame(index=data.index)
            for col in data.columns:
                if col != 'Period':
                    for k in range(1, lag + 1):
                        lagged_data[f'{col}_Lag_{k}'] = data[col].shift(k)
        #             lagged_data=lagged_data.drop(f'{col}_Lag_1',axis=1)
            return lagged_data


        lag = 3 # 
        # features = create_lag_features(gold_prices, lag)
        feature = gold_prices

        window_size = 43  # 

        
        modelss = {
            'TrendModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'SeasonalityModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'ResidualModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
                'TrendModel3': RandomForestRegressor(),
            'SeasonalityModel3': RandomForestRegressor(),
            'ResidualModel3': RandomForestRegressor(),
            'TrendModel4': LinearRegression(),
            'SeasonalityModel4': LinearRegression(),
            'ResidualModel4': LinearRegression()
        #         'Lasso': Lasso(0.5, fit_intercept=False)
        }

      
        # predictions = {model_name: [] for model_name in models}
        actual_price = []
        direction_correct = 0
        predictions=[]
        predictionsf=[]
        # features=features.fillna(method='bfill')
        # features=features.dropna()
        # print(features)
        for i in range(len(gold_prices['GoldPrice']) - window_size):

            # STL decomposition
            stl = STL(gold_prices['GoldPrice'].iloc[0:i + window_size])
            result = stl.fit()
            trend = result.trend
            seasonalo = result.seasonal
            residualo = result.resid
            newdata=seasonalo+residualo
            u, u_hat, omega = VMD(newdata, alpha, tau, K, DC, init, tol)
            u=u.astype(float)
            seasonal = pd.DataFrame(u[1],index=gold_prices.index[0:(i + window_size)])
            seasonal.columns = ['seasonal']
            residual = pd.DataFrame(u[0],index=gold_prices.index[0:(i + window_size)])
            residual.columns = ['residual']


            feature['Trend'] = trend
            feature['Seasonal'] = seasonal
            feature['Residual'] = residual
            # after creating features
        #     features = create_lag_features(feature, lag)
        #     names=['GoldPrice','TrendModel','SeasonalityModel','ResidualModel']
        #     features[names] = feature[names]
        #     print(feature.shape)

#            '''below delete external variables'''
#             featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
#             featureseasonal=feature.drop(['Trend','Residual'], axis=1)
#             featureresidual=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Trend'], axis=1)
            
            featuretrend=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
            featureseasonal=feature.drop(['Trend','Residual'], axis=1)
            featureresidual=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Trend'], axis=1)

            modelt=ARIMA(trend[0:i + window_size], order=(0, 1, 1))
            resultt = modelt.fit()
            trend_pred1 = resultt.forecast()[0]

            featuretrend = create_lag_features(featuretrend, lag)
            featuretrend['Trend']=feature['Trend']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtraint=featuretrend.iloc[lag:i + window_size,0:-1]
            ytraint=featuretrend.iloc[lag:i + window_size,-1]
            xtestt=featuretrend.iloc[i+window_size,0:-1]
        #     xtestt=np.array(xtestt).reshape(-1,featuretrend.shape[1]-1)

            featureseasonal = create_lag_features(featureseasonal, lag)
            featureseasonal['Seasonal']=feature['Seasonal']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrains=featureseasonal.iloc[lag:i + window_size,0:-1]
            ytrains=featureseasonal.iloc[lag:i + window_size,-1]
            xtests=featureseasonal.iloc[i+window_size,0:-1]
        #     xtests=np.array(xtests).reshape(-1,featureseasonal.shape[1]-1)

            models=ARIMA(residual[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            residual_pred1 = results.forecast()[0]

            featureresidual = create_lag_features(featureresidual, lag)
            featureresidual['Residual']=feature['Residual']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrainr=featureresidual.iloc[lag:i + window_size,0:-1]
            ytrainr=featureresidual.iloc[lag:i + window_size,-1]
            xtestr=featureresidual.iloc[i+window_size,0:-1]
        #     xtestr=np.array(xtestr).reshape(-1,featureresidual.shape[1]-1)

        #     scaler = MinMaxScaler()
        # #             scaler = StandardScaler()
        #     X_train = scaler.fit_transform(X_train)
        #     X_test = scaler.transform(X_test)
        #     scaler1 = MinMaxScaler()
        #     y_trend = scaler1.fit_transform(y_trend.values.reshape(-1, 1)).flatten()
        #     y_seasonal = scaler1.transform(y_seasonal.values.reshape(-1, 1)).flatten()
        #     y_residual = scaler1.transform(y_residual.values.reshape(-1, 1)).flatten()
            modelr=ARIMA(seasonal[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            resultr = modelr.fit()
            seasonal_pred1 = resultr.forecast()[0]

#           
            modelfs = RandomForestRegressor(random_state=10)
            modelfs.fit(xtraint,ytraint)
            importances = modelfs.feature_importances_
            threshold = np.median(importances)
            selected_features = xtraint.columns[importances > threshold]
            xtraint=xtraint[selected_features]
            xtestt=xtestt[selected_features]
            xtestt=np.array(xtestt).reshape(-1,xtraint.shape[1])


            modelfsr = RandomForestRegressor(random_state=10)
            modelfsr.fit(xtrainr,ytrainr)
            importancesr = modelfsr.feature_importances_
            thresholdr = np.median(importancesr)
            selected_featuresr = xtrainr.columns[importancesr > thresholdr]
            xtrainr=xtrainr[selected_featuresr]
            xtestr=xtestr[selected_featuresr]
            xtestr=np.array(xtestr).reshape(-1,xtrainr.shape[1])  

            modelfss = RandomForestRegressor(random_state=10)
            modelfss.fit(xtrains,ytrains)
            importancess = modelfss.feature_importances_
            thresholds = np.median(importancess)
            selected_featuress = xtrains.columns[importancess > thresholds]
            xtrains=xtrains[selected_featuress]
            xtests=xtests[selected_featuress]
            xtests=np.array(xtests).reshape(-1,xtrains.shape[1])       


            modelss['SeasonalityModel'].fit(xtrains, ytrains)
            modelss['TrendModel'].fit(xtraint, ytraint)
            modelss['ResidualModel'].fit(xtrainr, ytrainr)

            # forecasting
            trend_pred2 = modelss['TrendModel'].predict(xtestt)
            seasonal_pred2 = modelss['SeasonalityModel'].predict(xtests)
            residual_pred2 = modelss['ResidualModel'].predict(xtestr)

            modelss['SeasonalityModel3'].fit(xtrains, ytrains)
            modelss['TrendModel3'].fit(xtraint, ytraint)
            modelss['ResidualModel3'].fit(xtrainr, ytrainr)

#             modelss['SeasonalityModel4'].fit(xtrains, ytrains)
#             modelss['TrendModel4'].fit(xtraint, ytraint)
#             modelss['ResidualModel4'].fit(xtrainr, ytrainr)
            # 

            trend_pred3 = modelss['TrendModel3'].predict(xtestt)
            seasonal_pred3 = modelss['SeasonalityModel3'].predict(xtests)
            residual_pred3 = modelss['ResidualModel3'].predict(xtestr)
            
#             trend_pred4 = modelss['TrendModel4'].predict(xtestt)
#             seasonal_pred4 = modelss['SeasonalityModel4'].predict(xtests)
#             residual_pred4 = modelss['ResidualModel4'].predict(xtestr)

            #
        #     trend_pred = scaler1.inverse_transform(np.array(trend_pred).reshape(-1, 1)).flatten()
        #     seasonal_pred = scaler1.inverse_transform(np.array(seasonal_pred).reshape(-1, 1)).flatten()
        #     residual_pred = scaler1.inverse_transform(np.array(residual_pred).reshape(-1, 1)).flatten()
           
           
            predicted = trend_pred2+ seasonal_pred1 + residual_pred2
#             predicted = (trend_pred2+trend_pred3)/2 +seasonal_pred1 + (residual_pred2+residual_pred3)/2
#             predicted = (trend_pred2+ seasonal_pred1 + residual_pred2+trend_pred1+ seasonal_pred1 + residual_pred1+trend_pred3+ seasonal_pred1 + residual_pred3)/3
            actual_price.append(gold_prices['GoldPrice'].iloc[i + window_size])

              

            if i < len(feature) - window_size:
                    actual_change = gold_prices['GoldPrice'].iloc[i + window_size ] - gold_prices['GoldPrice'].iloc[i + window_size -1]
                    predicted_change = predicted - gold_prices['GoldPrice'].iloc[i + window_size -1]
                    if np.sign(actual_change) == np.sign(predicted_change):
                        direction_correct += 1

            predictions.append(predicted)
        predictions = np.concatenate([arr for arr in predictions])
        actual_price = np.array(actual_price)
        '''weighting ensemble'''
#         gold_prices=pd.read_csv("gold.csv")
        var=pd.read_csv('varpre-1step12test.csv')
        gold_prices=gold_prices.iloc[:,0:]
#         gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
#         gold_prices.set_index('Period',inplace=True)
        
        predic=np.zeros([len(gold_prices['GoldPrice']) - window_size,1])
        # predicted=[]
        # print(var.iloc[i,0])
        for i in range(len(actual_price)):

            if i==0:
                predic[0]=(predictions[i]+var.iloc[i,1])/2
            elif i>0:
                if np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100>4 and np.abs((actual_price[i-1]-var.iloc[i-1,1]) /actual_price[i-1])*100>4:
            #                     predicted=(predictions[i]+var.iloc[i,1])/2
                    if np.abs(actual_price[i-1]-predictions[i-1])>np.abs(actual_price[i-1]-var.iloc[i-1,1]):
                        predicted=var.iloc[i,1]
                    else:
                        predicted=predictions[i]
#                 elif np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100>4 and np.abs((actual_price[i-1]-var.iloc[i-1,1]) /gold_prices.iloc[window_size+i,0])*100<=4:
#                     predicted=(predictions[i]+var.iloc[i,1])/2
#                 elif np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100<=4 and np.abs((actual_price[i-1]-var.iloc[i-1,1]) /gold_prices.iloc[window_size+i,0])*100>4:
#                     predicted=(predictions[i]+var.iloc[i,1])/2
                else:
                    a=1/np.abs(actual_price[i-1]-predictions[i-1])
                    b=1/np.abs(actual_price[i-1]-var.iloc[i-1,1])
                    predicted=(a/(a+b))*predictions[i]+(b/(a+b))*var.iloc[i,1]

#             predic[i]=predicted


                    
                    
                    
                    
            #set parameters
#                     pop = 50 #
#                     MaxIter = 500 #
#                     dim = 2 #
#                     lb = 0*np.ones(dim) #
#                     ub = 1*np.ones(dim)#
#                     # fitness function selection
#                     fobj = fun
#                     GbestScore,GbestPositon,Curve = TLBO(pop,dim,lb,ub,MaxIter,fobj,predictions[i-1],var.iloc[i-1,1],gold_prices.iloc[window_size+i,0]) 
#                     GbestPositonnn = [item for sublist in GbestPositon for item in sublist]
#                     a=GbestPositonnn[0]
#                     b=1-GbestPositonnn[0]
#                     print('best fitness', GbestScore)

#                     predicted=a*predictions[i]+b*var.iloc[i,1]
#                     print(predicted)
            predic[i]=predicted

        
        predictionsf = [item for sublist in predic for item in sublist]
        '''end'''
       
        tmp=0
        for loop in range(1,len(actual_price)):
            if(( predictionsf[loop] - actual_price[loop-1 ]) * (actual_price[loop] - actual_price[loop-1])>=0):
                tmp=tmp+1;        
        Dstat = tmp / (len(actual_price)-1)
        
        '''D'''
        # 计
        mse = mean_squared_error(actual_price[1:], predictionsf[1:])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual_price[1:], predictionsf[1:])
        mape = np.mean(np.abs((actual_price[1:] - predictionsf[1:]) / actual_price[1:])) * 100
#         total_directions = len(feature) - window_size 
        
   
#         direction_accuracy = (direction_correct/ total_directions) * 100

        metrics = {
                'RMSE': rmse,
                'MSE': mse,
                'MAE': mae,
                'MAPE':mape,
                'Directional Accuracy': Dstat,
                'actual_price':actual_price,
                'predictions':predictions,
                'var':np.array(var.iloc[0:,1]),
                'predictionsf':predictionsf
            }
        print(metrics)


        # df_metrics = pd.DataFrame(metrics)
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        print(actual_price)
        print(predictionsf)
        df_metrics = pd.DataFrame(metrics)
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}', index=metrics)

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
# from vmdpy import VMD  
#. some sample parameters for VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
K =3            # 3 modes  
DC = 0             # no DC part imposed  
init = 1           # initialize omegas uniformly  
tol = 1e-7  

import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import( LinearRegression, 
                                 ElasticNet, 
                                 Ridge, 
                                 Lasso, 
                                 HuberRegressor)
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
gold_prices=pd.read_csv("gold.csv")
# varpre=pd.read_csv('varpre-1step12test.csv')
gold_prices=gold_prices.iloc[:,0:]
gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
gold_prices.set_index('Period',inplace=True)
with pd.ExcelWriter('demand-proposed.xlsx') as writer:
    for r in range(10):  
        def create_lag_features(data, lag):
            lagged_data = pd.DataFrame(index=data.index)
            for col in data.columns:
                if col != 'Period':
                    for k in range(1, lag + 1):
                        lagged_data[f'{col}_Lag_{k}'] = data[col].shift(k)
        #             lagged_data=lagged_data.drop(f'{col}_Lag_1',axis=1)
            return lagged_data


        lag = 3 # 
        # features = create_lag_features(gold_prices, lag)
        feature = gold_prices

        window_size = 43 # 

   
        modelss = {
            'TrendModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'SeasonalityModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'ResidualModel': BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100),
            'lasso1':RandomForestRegressor(),
            'lasso2': RandomForestRegressor(),
            'lasso3':RandomForestRegressor()
        }

   
        # predictions = {model_name: [] for model_name in models}
        actual_price = []
        direction_correct = 0
        predictions=[]
        predictionsf=[]
        # features=features.fillna(method='bfill')
        # features=features.dropna()
        # print(features)
        for i in range(len(gold_prices['GoldPricede']) - window_size):


            u, u_hat, omega = VMD(gold_prices['GoldPricede'].iloc[0:i + window_size], alpha, tau, K, DC, init, tol)
            u=u.astype(float)
            trend = pd.DataFrame(u[0],index=gold_prices.index[0:(i + window_size)])
            trend.columns = ['trend']

            newdata = pd.DataFrame(gold_prices['GoldPricede'].iloc[0:i + window_size]-u[0],index=gold_prices.index[0:(i + window_size)])
            stl = STL(newdata)
            result = stl.fit()
            seasonal = result.seasonal+result.resid
            residual = result.trend


            feature['Trend'] = trend
            feature['Seasonal'] = seasonal
            feature['Residual'] = residual
          
        #     features = create_lag_features(feature, lag)
        #     names=['GoldPrice','TrendModel','SeasonalityModel','ResidualModel']
        #     features[names] = feature[names]
        #     print(feature.shape)

        #     print(features.shape)
            featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
            featureseasonal=feature.drop(['GoldPricede','MAdemand','diffdemand','Trend','Residual'], axis=1)
            featureresidual=feature.drop(['GoldPricede','MAdemand','diffdemand','Seasonal','Trend'], axis=1)
            #'''no external variables'''
#             featuretrend=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Residual'], axis=1)
#             featureseasonal=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Trend','Residual'], axis=1)
#             featureresidual=feature.drop(['GoldPrice','GoldPricede','MAgold','diffgold','MAdemand','diffdemand','Seasonal','Trend'], axis=1)

            modelt=ARIMA(trend[0:i + window_size], order=(0, 1, 1))
            resultt = modelt.fit()
            trend_pred1 = resultt.forecast()[0]

            featuretrend = create_lag_features(featuretrend, lag)
            featuretrend['Trend']=feature['Trend']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtraint=featuretrend.iloc[lag:i + window_size,0:-1]
            ytraint=featuretrend.iloc[lag:i + window_size,-1]
            xtestt=featuretrend.iloc[i+window_size,0:-1]
            xtestt=np.array(xtestt).reshape(-1,featuretrend.shape[1]-1)

            featureseasonal = create_lag_features(featureseasonal, lag)
            featureseasonal['Seasonal']=feature['Seasonal']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrains=featureseasonal.iloc[lag:i + window_size,0:-1]
            ytrains=featureseasonal.iloc[lag:i + window_size,-1]
            xtests=featureseasonal.iloc[i+window_size,0:-1]
        #     xtests=np.array(xtests).reshape(-1,featureseasonal.shape[1]-1)

            models=ARIMA(seasonal[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            seasonal_pred1 = results.forecast()[0]

            featureresidual = create_lag_features(featureresidual, lag)
            featureresidual['Residual']=feature['Residual']
        #     featuretrend = featuretrend.drop(featuretrend.head(lag).index)
            xtrainr=featureresidual.iloc[lag:i + window_size,0:-1]
            ytrainr=featureresidual.iloc[lag:i + window_size,-1]
            xtestr=featureresidual.iloc[i+window_size,0:-1]


            models=ARIMA(residual[0:i + window_size], order=(0, 1, 1))
        #     models=SARIMAX(seasonal[0:i + window_size], order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
            results = models.fit()
            residual_pred1 = results.forecast()[0]

        #     xtestr=np.array(xtestr).reshape(-1,featureresidual.shape[1]-1)

        #     scaler = MinMaxScaler()
        # #             scaler = StandardScaler()
        #     X_train = scaler.fit_transform(X_train)
        #     X_test = scaler.transform(X_test)
        #     scaler1 = MinMaxScaler()
        #     y_trend = scaler1.fit_transform(y_trend.values.reshape(-1, 1)).flatten()
        #     y_seasonal = scaler1.transform(y_seasonal.values.reshape(-1, 1)).flatten()
        #     y_residual = scaler1.transform(y_residual.values.reshape(-1, 1)).flatten()

       
            modelfs = RandomForestRegressor(random_state=0)
            modelfs.fit(xtrains,ytrains)
            importances = modelfs.feature_importances_
            threshold = np.median(importances)
            selected_features = xtrains.columns[importances > threshold]
            xtrains=xtrains[selected_features]
            xtests=xtests[selected_features]
            xtests=np.array(xtests).reshape(-1,xtrains.shape[1])

        
            modelfsr = RandomForestRegressor(random_state=0)
            modelfsr.fit(xtrainr,ytrainr)
            importancesr = modelfsr.feature_importances_
            thresholdr = np.median(importances)
            selected_featuresr = xtrainr.columns[importancesr > thresholdr]
            xtrainr=xtrainr[selected_featuresr]
            xtestr=xtestr[selected_featuresr]
            xtestr=np.array(xtestr).reshape(-1,xtrainr.shape[1])  

            modelss['SeasonalityModel'].fit(xtrains, ytrains)
            modelss['TrendModel'].fit(xtraint, ytraint)
            modelss['ResidualModel'].fit(xtrainr, ytrainr)

            modelss['lasso2'].fit(xtrains, ytrains)
            modelss['lasso1'].fit(xtraint, ytraint)
            modelss['lasso3'].fit(xtrainr, ytrainr)
            # 

            trend_pred2 = modelss['TrendModel'].predict(xtestt)
            seasonal_pred2 = modelss['SeasonalityModel'].predict(xtests)
            residual_pred2 = modelss['ResidualModel'].predict(xtestr)
            
            trend_pred3 = modelss['lasso1'].predict(xtestt)
            seasonal_pred3 = modelss['lasso2'].predict(xtests)
            residual_pred3 = modelss['lasso3'].predict(xtestr)

            #
        #     trend_pred = scaler1.inverse_transform(np.array(trend_pred).reshape(-1, 1)).flatten()
        #     seasonal_pred = scaler1.inverse_transform(np.array(seasonal_pred).reshape(-1, 1)).flatten()
        #     residual_pred = scaler1.inverse_transform(np.array(residual_pred).reshape(-1, 1)).flatten()
            
# 
            predicted = trend_pred1+ seasonal_pred2 + residual_pred2
#             predicted = (trend_pred1+ seasonal_pred2 + residual_pred2+trend_pred1+ seasonal_pred3 + residual_pred3)/2

        
            actual_price.append(gold_prices['GoldPricede'].iloc[i + window_size])
#             predicted=0.9*predicted+0.1*varpre.iloc[i,2]    
            #     predicted=0.98*predicted+0.02*varpre.iloc[i+1,2] 
#             if i < len(feature) - window_size:
#                     actual_change = gold_prices['GoldPricede'].iloc[i + window_size ] - gold_prices['GoldPricede'].iloc[i + window_size -1]
#                     predicted_change = predicted - gold_prices['GoldPricede'].iloc[i + window_size -1]
#                     if np.sign(actual_change) == np.sign(predicted_change):
#                         direction_correct += 1

            predictions.append(predicted)
        predictions = np.concatenate([arr for arr in predictions])
        actual_price = np.array(actual_price)
        '''weight ensemble'''
#         gold_prices=pd.read_csv("gold.csv")
        var=pd.read_csv('varpre-1step12test.csv')
        gold_prices=gold_prices.iloc[:,0:]
#         gold_prices['Period']=pd.to_datetime(gold_prices['Period'])
#         gold_prices.set_index('Period',inplace=True)
        
        predic=np.zeros([len(gold_prices['GoldPrice']) - window_size,1])
        # predicted=[]
        # print(var.iloc[i,0])
        for i in range(len(actual_price)):

            if i==0:
                predic[0]=(predictions[i]+var.iloc[i,2])/2
            elif i>0:
                if np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100>10 and np.abs((actual_price[i-1]-var.iloc[i-1,2]) /actual_price[i-1])*100>10:
            #                     predicted=(predictions[i]+var.iloc[i,1])/2
                    if np.abs(actual_price[i-1]-predictions[i-1])>np.abs(actual_price[i-1]-var.iloc[i-1,2]):
                        predicted=var.iloc[i,2]
                    else:
                        predicted=predictions[i]
#                 elif np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100>10 and np.abs((actual_price[i-1]-var.iloc[i-1,2]) /gold_prices.iloc[window_size+i,0])*100<=10:
#                     predicted=(predictions[i]+var.iloc[i,2])/2
#                 elif np.abs((actual_price[i-1]-predictions[i-1]) / actual_price[i-1]) * 100<=10 and np.abs((actual_price[i-1]-var.iloc[i-1,2]) /gold_prices.iloc[window_size+i,0])*100>10:
#                     predicted=(predictions[i]+var.iloc[i,2])/2
                else:
                    a=1/np.abs(actual_price[i-1]-predictions[i-1])
                    b=1/np.abs(actual_price[i-1]-var.iloc[i-1,2])
                    predicted=(a/(a+b))*predictions[i]+(b/(a+b))*var.iloc[i,2]

#             predic[i]=predicted


                    
                    
                    
                    
            #set parameters
#                     pop = 50 #
#                     MaxIter = 500 
#                     dim = 2 #
#                     lb = 0*np.ones(dim) 
#                     ub = 1*np.ones(dim)
#                
#                     fobj = fun
#                     GbestScore,GbestPositon,Curve = TLBO(pop,dim,lb,ub,MaxIter,fobj,predictions[i-1],var.iloc[i-1,1],gold_prices.iloc[window_size+i,0]) 
#                     GbestPositonnn = [item for sublist in GbestPositon for item in sublist]
#                     a=GbestPositonnn[0]
#                     b=1-GbestPositonnn[0]
#                     print('bet fitness', GbestScore)

#                     predicted=a*predictions[i]+b*var.iloc[i,1]
#                     print(predicted)
            predic[i]=predicted

        
        predictionsf = [item for sublist in predic for item in sublist]
        '''end'''
      
        tmp=0
        for loop in range(1,len(actual_price)):
            if(( predictionsf[loop] - actual_price[loop-1 ]) * (actual_price[loop] - actual_price[loop-1])>=0):
                tmp=tmp+1;        
        Dstat = tmp / (len(actual_price)-1)
        

        mse = mean_squared_error(actual_price[1:], predictionsf[1:])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual_price[1:], predictionsf[1:])
        mape = np.mean(np.abs((actual_price[1:] - predictionsf[1:]) / actual_price[1:])) * 100
#         total_directions = len(feature) - window_size 
        

#         direction_accuracy = (direction_correct/ total_directions) * 100

        metrics = {
                'RMSE': rmse,
                'MSE': mse,
                'MAE': mae,
                'MAPE':mape,
                'Directional Accuracy': Dstat,
                'actual_price':actual_price,
                'predictions':predictions,
                'var':np.array(var.iloc[0:,2]),
                'predictionsf':predictionsf
            }
        print(metrics)


        # df_metrics = pd.DataFrame(metrics)
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        print(actual_price)
        print(predictionsf)
        df_metrics = pd.DataFrame(metrics)
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}', index=metrics)